In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from typing import List
from ipynb.fs.full.summarize_docs import new_summarize_api, read_docx  # Import functions from summarizer.py
import nest_asyncio
nest_asyncio.apply()
import uvicorn


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  # Replace with your frontend URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/summarize/")
async def summarize(files: List[UploadFile] = File(...)):
    combined_content = ""
    file_counter=0
    
    try:
        # Read content from each file and combine
        for file in files:
            file_counter += 1
            print(1)
            content = read_docx(file)
            print(2)
            combined_content +=  '**** Experiment '+str(file_counter) +': \n' + content + "\n"  # Concatenate contents with a newline
            # file.file.seek(0)

        print(combined_content)
        # Generate a summary for the combined content
        summary = new_summarize_api(combined_content)
        
        return {"summary": summary}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()


INFO:     Started server process [13052]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55382 - "GET / HTTP/1.1" 404 Not Found
